In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle

#from calclation import RMSProp
from calclation import Adam
from network import main_network

import data_augmentation

### Numpyを用いてデータ拡張をおこない学習する

data_augmentation.pyにまとめてあります。
random_erasingについては、実装したもののあまり効果が得られず使いませんでした

In [20]:
train_data = np.load("/Users/sawaki/skill-up-ai/train_data.npy")
train_label = np.load("/Users/sawaki/skill-up-ai/train_label.npy")
train_data = train_data.astype('float32')
train_data = train_data / train_data.max()

In [30]:
epochs = 4
batch_size = 32
loop = 4


In [5]:
# network
optimizer = Adam()
network = main_network()

In [22]:
# loss/accuracy
train_loss = []
test_loss = []
train_acc = []
test_acc = []

In [35]:
for i in range(loop):
    #split data into train/test 
    train, test, train_labels, test_labels = train_test_split(train_data, train_label, test_size=0.3,shuffle=True)

    #data augmentation by rotating image
    #train = data_augmentation.rotation_crop(train, angle_range=(-20, 20))
    train = data_augmentation.redpad_rotation_crop(train, angle_range=(-15, 15))

    train = train.reshape(-1, 1, 28, 28)
    test = test.reshape(-1, 1, 28, 28)
    x = train
    t = train_labels

    # iteration
    xsize = x.shape[0]
    #print(xsize)
    iteration = np.ceil(xsize / batch_size).astype(np.int)

    #x = x.reshape(-1,1,28,28)
    for epoch in range(epochs):
        print("epoch = {0}-{1}".format(i,epoch))

        # shuffling data
        index = np.arange(xsize)
        np.random.shuffle(index)
        #print(index)

        for iter in range(iteration):
            #print("iter=", iter)
            # Extract the corresponding index
            corr_index = index[batch_size*iter : batch_size*(iter+1)]

            # mini batch
            x_train = x[corr_index]
            t_train = t[corr_index]


            batches = x_train

            #print(batches.shape, t_train.shape)

            # forward/back propagation 
            grads = network.gradient(batches, t_train)

            # update
            optimizer.update(network.params, grads)



        # traindata
        # calc loss   
        train_loss.append(network.loss(x,  t))
        # calc acc
        train_acc.append(network.accuracy(x, t))

        # testdata
        # calc loss
        test_loss.append(network.loss(test, test_labels))
        # calc acc
        test_acc.append(network.accuracy(test, test_labels))
        print(network.accuracy(test, test_labels))
        if network.accuracy(test, test_labels) > 0.993:
            with open("katakana_model_K{0}.pickle".format(network.accuracy(test, test_labels)), "wb") as f:
                pickle.dump(network, f)

/var/folders/v6/0z_kylj93ll5z4_htn4grf000000gn/T/ipykernel_5670/3072916224.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  iteration = np.ceil(xsize / batch_size).astype(np.int)
/Users/sawaki/miniforge3/envs/pose/lib/python3.8/site-packages/keras_preprocessing/image/numpy_array_iterator.py:129: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (32, 1, 28, 28) (28 channels).
  warnings.war

epoch = 0-0


/Users/sawaki/miniforge3/envs/pose/lib/python3.8/site-packages/keras_preprocessing/image/numpy_array_iterator.py:129: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (20, 1, 28, 28) (28 channels).
  warnings.warn('NumpyArrayIterator is set to use the '


0.9744444444444444
epoch = 0-1
0.9844444444444445
epoch = 0-2
0.9933333333333333
epoch = 0-3
0.9922222222222222
epoch = 1-0
0.9911111111111112
epoch = 1-1
0.9922222222222222
epoch = 1-2
0.9944444444444445
epoch = 1-3
0.9977777777777778
epoch = 2-0
0.9988888888888889
epoch = 2-1
0.9855555555555555
epoch = 2-2
0.9722222222222222
epoch = 2-3
0.9977777777777778
epoch = 3-0
0.9988888888888889
epoch = 3-1
1.0
epoch = 3-2
1.0
epoch = 3-3
0.9977777777777778


In [36]:
print(train_acc)

[0.9914285714285714, 0.9961904761904762, 0.9923809523809524, 0.9942857142857143, 0.9985714285714286, 0.9995238095238095, 0.9957142857142857, 0.9957142857142857, 0.9976190476190476, 0.991904761904762, 0.9942857142857143, 0.9995238095238095, 0.9971428571428571, 0.9985714285714286, 0.9814285714285714, 0.9976190476190476, 0.9961904761904762, 0.9971428571428571, 0.9980952380952381, 0.9990476190476191, 0.9957142857142857, 0.9966666666666667, 0.9957142857142857, 0.9990476190476191, 0.9947619047619047, 0.9980952380952381, 0.9990476190476191, 0.9833333333333333, 0.9985714285714286, 0.9976190476190476, 0.9976190476190476, 0.9995238095238095, 0.9980952380952381, 0.9980952380952381, 0.9976190476190476, 0.9938095238095238, 0.9971428571428571, 0.9985714285714286, 0.9980952380952381, 0.9990476190476191, 0.9995238095238095, 0.9976190476190476, 0.9971428571428571, 0.9995238095238095, 1.0, 0.9995238095238095, 0.9985714285714286, 0.9957142857142857, 0.9980952380952381, 0.9961904761904762, 0.9985714285714

In [37]:
print(test_acc)

[0.9833333333333333, 0.9944444444444445, 0.9811111111111112, 0.9822222222222222, 0.9955555555555555, 0.9911111111111112, 0.9811111111111112, 0.9911111111111112, 0.9933333333333333, 0.9888888888888889, 0.9833333333333333, 0.9955555555555555, 0.9966666666666667, 0.9944444444444445, 0.9722222222222222, 0.9922222222222222, 0.9944444444444445, 0.9955555555555555, 0.9955555555555555, 0.9955555555555555, 0.9911111111111112, 0.99, 0.9955555555555555, 0.9955555555555555, 0.9944444444444445, 0.9988888888888889, 0.9955555555555555, 0.98, 0.9944444444444445, 0.9944444444444445, 0.9911111111111112, 0.9922222222222222, 0.9988888888888889, 0.9977777777777778, 0.9922222222222222, 0.9922222222222222, 0.9866666666666667, 0.9933333333333333, 0.9944444444444445, 0.9966666666666667, 0.9933333333333333, 0.9944444444444445, 0.9955555555555555, 0.9955555555555555, 0.9944444444444445, 0.9955555555555555, 0.9888888888888889, 0.9866666666666667, 0.99, 0.9933333333333333, 0.9933333333333333, 0.9944444444444445, 0

In [34]:
test_acc_rev = list(reversed(test_acc))
#print(test_acc_rev)

num = 5
sum = 0.0
acc_ave = 0.0
for i in range(num):
    sum += test_acc_rev[i]

acc_ave = sum/num    
print(acc_ave)


0.9773333333333334


epochs = 8, 
batch_size = 32, 
lr = 0.01, 
loop = 8, 
angle_range=(-15, 15)

- average of test_acc (last 5 epoch) is 0.9748 

epochs = 8, 
batch_size = 32, 
lr = 0.01, 
loop = 8, 
angle_range=(-15, 15)
random erasing

- average of test_acc (last 5 epoch) is 0.9593

epochs = 8
batch_size = 32
lr = 0.01
loop = 8
angle_range=(-10, 10)

- average of test_acc (last 5 epoch) is 0.9724 

epochs = 8
batch_size = 32
lr = 0.01
loop = 8
angle_range=(-20, 20)

- average of test_acc (last 5 epoch) is 0.9566 


epochs = 8
batch_size = 32
lr = 0.01
loop = 8
angle_range=(-10, 10)
クッキリあり

- average of test_acc (last 5 epoch) is 0.9073 

epochs = 4,
batch_size = 32,
lr = 0.01,
loop = 16,
angle_range=(-15, 15)

- average of test_acc (last 5 epoch) is 0.9631 

In [38]:
with open("katakana_model_best2.pickle", "wb") as f:
    pickle.dump(network, f)